# 최종 모의고사
***

## 1-1. Salary를 정규화 결과가 0.2 이상 0.4 미만인 데이터의 Age의 평균과 정규화 결과가 0.6 초과 0.75 미만인 데이터의 Age 평균의 차이(절댓값)를 구하시오.

In [12]:
%%time

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
train = pd.read_csv('X_train.csv')
scaler = MinMaxScaler()
train['ms_1'] = scaler.fit_transform(train[['Salary']])
r1 = train.query('0.2 <= ms_1 < 0.4').Age.mean()
r2 = train.query('0.6 < ms_1 < 0.75').Age.mean()
print(abs(r2 - r1))

2.683302543380343
CPU times: user 24.8 ms, sys: 3.6 ms, total: 28.4 ms
Wall time: 25.8 ms


## 1-2. Dateofjoining에서 월(month)을 추출하고 Gender별 Salary와 월의 상관계수를 구한 후 두 값의 곱을 구하시오.

In [14]:
%%time
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
train = pd.read_csv('X_train.csv')
train['month'] = pd.to_datetime(train['Dateofjoining']).dt.month
v1 = train[train.Gender == 'Male'].corr()['Salary']['month']
v2 = train[train.Gender == 'Female'].corr()['Salary']['month']
print(v1 * v2)

0.0005773552105973039
CPU times: user 22.4 ms, sys: 2.65 ms, total: 25.1 ms
Wall time: 22.9 ms


## 1-3. Quarterly Rating을 예측하는 모델링을 수행하시오. 이 때 예측확률을 도출하고 result1이라는 파일명으로 submission을 저장하시오.

In [89]:
submission.head()

,ID,Quarterly Rating_1,Quarterly Rating_2,Quarterly Rating_3,Quarterly Rating_4
0,1,0,0,0,0
1,2,0,0,0,0
2,3,0,0,0,0
3,4,0,0,0,0
4,5,0,0,0,0


In [26]:
%%time
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')
submission = pd.read_csv("submission.csv")

X_train = X_train.iloc[:, 1:]
X_test = X_test.iloc[:, 1:]

X_train.drop('City', axis = 1, inplace = True)
X_test.drop('City', axis = 1, inplace = True)

X_train['Gender'] = [1 if g == 'Male' else 0 for g in X_train['Gender']]
X_test['Gender'] = [1 if g == 'Male' else 0 for g in X_test['Gender']]

X_train['month'] = pd.to_datetime(X_train.Dateofjoining).dt.month
X_test['month'] = pd.to_datetime(X_test.Dateofjoining).dt.month

X_train.drop('Dateofjoining', axis = 1, inplace = True)
X_test.drop('Dateofjoining', axis = 1, inplace = True)

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

X = X_train.copy()
y = y_train.iloc[:, -1]

rf_losses = []
xgb_losses = []

xgb_pred = np.zeros((X_test.shape[0], 4))
rf_pred = np.zeros((X_test.shape[0], 4))

for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    rf = RandomForestClassifier(random_state = 42, max_depth = 4)
    xgb = XGBClassifier(random_state = 42, max_depth = 4)
    
    rf.fit(tr_x, tr_y)
    xgb.fit(tr_x, tr_y)
    
    val_rf = rf.predict_proba(val_x)
    val_xgb = xgb.predict_proba(val_x)
    
    rf_loss = log_loss(val_y, val_rf)
    xgb_loss = log_loss(val_y, val_xgb)
    
    rf_losses.append(rf_loss)
    xgb_losses.append(xgb_loss)
    
    rf_fold = rf.predict_proba(X_test) / 10
    xgb_fold = xgb.predict_proba(X_test) / 10
    
    rf_pred += rf_fold
    xgb_pred += xgb_fold
    
    
print(f'{rf.__class__.__name__}의 평균 logloss는 {np.mean(rf_losses)}')
print(f'{xgb.__class__.__name__}의 평균 logloss는 {np.mean(xgb_losses)}')

submission.iloc[:, 1:] = (rf_pred + xgb_pred) / 2
submission.to_csv('result1.csv', index = False)

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:33:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:33:05] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[12:33:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:33:09] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:33:09] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RandomForestClassifier의 평균 logloss는 0.7968072443558092
XGBClassifier의 평균 logloss는 0.8028575971390431
CPU times: user 33 s, sys: 5.63 s, total: 38.7 s
Wall time: 5.33 s
